In [79]:
import pandas as pd
from elftools.elf.elffile import ELFFile

In [80]:
config = "fdsc_inter-rasm"
old_csv = "~/faults_" + config + "_new.csv"
new_csv = "faults_" + config + "_names.csv"
bin_name = "../out_" + config + "_1.elf"

data = pd.read_csv(old_csv)
data

,attempt,stop_addr,stop_fn,delay,target,bitflip,code,seed
0,0,0x0801740a,Reset_Handler,0.006,0x20005596,4,0,3735928559
1,1,0x0800d432,vTaskTimerTest,0.985,0x200027bc,1,0,3735928559
2,2,0x08015804,pxPortInitialiseStack,0.056,0x2000065b,128,0,3735928559
3,3,0x0800dca6,crc32buf_ret_dup,0.898,0x20000604,16,0,3735928559
4,4,0x0800caa0,Multiply_dup,0.495,0x20005caa,32,0,3735928559
...,...,...,...,...,...,...,...,...
59995,59995,0x0800cace,Multiply_dup,0.525,0x20004396,8,0,3735928559
59996,59996,0x08007906,xQueueGenericSend_ret_dup,0.925,0x20005011,64,0,3735928559
59997,59997,0x08003350,prvInsertTimerInActiveList_ret_dup,0.301,0x200040f3,16,0,3735928559
59998,59998,0x0800dd64,crc32buf_ret_dup,0.325,0x20000538,64,0,3735928559


In [81]:
symtable = dict()

with open(bin_name, 'rb') as elffile:
    for section in ELFFile(elffile).iter_sections():
        try:
            for sym in section.iter_symbols():
                if (sym.name != "ucHeap" and sym.entry['st_info']['type'] == 'STT_OBJECT'):
                    symtable[sym.name] = {"addr": sym.entry['st_value'], "size": sym.entry['st_size']}
        except: 
            continue

In [82]:
data_size = data.shape[0]

new_data = [None for i in range(data_size)]

def associate_symbols():

    for idx, row in data.iterrows():
        found = False
        for sym in symtable.keys():
            try:
                if int(row['target'][2:], base=16) in range(symtable[sym]['addr'], symtable[sym]['addr']+symtable[sym]['size'], 1):
                    #print(symtable[sym])
                    new_data[idx] = sym
                    break
            except:
                break
        print(str(idx) + "/" + str(data_size), end = "\r")

associate_symbols()

data['struct'] = new_data

In [83]:
data.to_csv(new_csv)

In [84]:
'''
    37: 20001900  2000 OBJECT  LOCAL  DEFAULT   10 main.xBufferRecvStack
    34: 20001068  2000 OBJECT  LOCAL  DEFAULT   10 main.xTaskStack
'''

print(hex(0x20001900 + 2000))
print(hex(0x20001068 + 2000))

0x200020d0
0x20001838
